# Amazon RDS 기초

## 관계형 데이터베이스
- 데이터의 관계에 집중한 데이터베이스
    - 사전에 정의된 관계가 있을 때 사용
- 미리 지정된 형식과 타입의 데이터만 저장 가능
- 테이블 형식으로 데이터를 관리
    - 행과 열을 기반으로 한 여러 테이블을 통해 데이터를 정의
- 고유의 키로 각 데이터를 식별
- 트랜젝션 지원
    - 원하는 동작이 정확히 수행되거나 완전히 실패 둘 중 하나로 유지
    - 즉 어중간한 상태(일부만 반영되거나 실패했으나 데이터 변경이 되거나)는 없음
- 사용 사례 : 일반적인 어플리케이션, 온라인 게임 등

## RDS
- RDS를 사용하면 클라우드에서 관계형 데이터베이스를 간편하게 설정, 운영 및 확장할 수 있습니다. 하드웨어 프로비저닝, 데이터베이스 설정, 패치 및 백업과 같은 시간 소모적인 관리 작업을 자동화하면서 비용 효율적이고 크기 조정 가능한 용량을 제공합니다. 사용자가 어플리케이션에 집중하여 어플리케이션에 필요한 빠른 성능, 고가용성, 보안 및 호환성을 제공할 수 있도록 지원합니다.
- 관계형 데이터베이스를 제공하는 서비스
    - Relational Database Service : 관계형 데이터베이스
    - <-> NoSql(DynamoDB,DocumentDB, ElasticCache)
- 가상 머신 위에서 동작(EC2와 비슷한 인스턴스에서 동작)
    - 단, 직접 시스템에 로그인 불가능 -> OS 패치, 관리 등은 AWS의 역할
- 서버리스 서비스가 아님
    - 단, Aurora Serverless는 서버리스 서비스가 맞음
- 암호화 지원
- 자동 백업 지원

## RDS와 EC2
- 내부에서는 EC2를 활용
- VPC 안에서 동작
    - 기본적으로 Public IP를 부여하지 않으면 외부에서 접근 불가능
    - 설정에 따라 Public IP를 부여하여 인터넷에서 접근 가능(DNS로 접근)
- 서브넷과 보안그룹 지정 필요
- EC2 타입의 지정 필요
- 스토리지는 EBS를 활용
    - EBS 타입 및 용량 선택 필요
- 중지 가능
    - 단 7일 후에 자동으로 다시 시작됨
- 백업으로 스냅샷 생성 가능
- Reserved Instance 활용 가능
    - 일정 기간을 약정하여 최대 70% 정도 할인

## RDS의 인증 방법
- 전통적인 아이디/패스워드 방식
    - AWS Secrets Manager와 연동하여 자동 로테이션 가능
- IAM DB 인증
    - 데이터베이스를 IAM 유저 자격증명 / 역할을 통해 관리 가능
- Kerberos 인증

## RDS에서 제공하는 DB 엔진
- MS SQL Server
- Oracle
    - Oracle OLAP
- MySQL Server
- PostgreSQL
- Maria DB
- IBM Db2
- Amazon Aurora

## RDS Multi AZ
- 두 개 이상의 AZ에 걸쳐 데이터베이스를 구축하고 원본과 다른 DB(Standby)를 자동으로 동기화(Sync)
    - 원본 DB에 쓰기를 하면, StandBy DB로 데이터를 전송. 둘 다 쓰기가 완료되면 유저에게 "완료" 응답
- 원본 DB의 장애 발생 시 자동으로 다른 DB가 원본으로 승격됨(DNS가 Stanby DB로 라우팅)
- Stanby DB는 직접 접근 불가능
- 퍼포먼스의 상승 효과가 아닌 안정성을 위한 서비스

## 읽기 전용 복제본
- 데이터베이스의 읽기 전용 복제본을 생성(Async)
    - Primary DB에 쓰기를 하면, 완료되자 마자 유저에게 "완료"를 응답. 이 후에 비동기로 Replica DB에 데이터를 전송
- 쓰기는 원본 데이터베이스에, 읽기는 복제본에서 처리하여 워크로드 분산
- 안정성이 아닌 퍼포먼스를 위한 서비스
- 원본 DB의 장애 발생 시 수동으로 DNS 변경이 필요함

## RDS Multi Region
- 다른 리전에 지속적으로 동기화 시키는 DB 클러스터를 생성
    - Async 복제
- 주로 로컬 퍼포먼스 혹은 DR 시나리오로 활용

## DB Subnet Group
- RDS가 프로비전되는 서브넷을 묶은 그룹
    - 실전에서는 주로 프라이빗 서브넷만을 사용(보안적인 이유)
    - 같은 리전 내 서브넷이 최소 두개 이상 필요
- 퍼블릭 접근을 허용하기 위해서는 퍼블릭 서브넷으로 구성 필요

## Parameter Group
- 데이터베이스의 주요 파라미터(타임존, 패스워드 유효시간, 기본 CharSet등)를 묶은 논리적 단위
- 미리 지정해 둔 파라미터 설정 모음으로 여러 RDS에 적용 가능